In [1]:
import os
from tqdm import tqdm
import re
import numpy as np
from collections import Counter, defaultdict

from scipy.sparse import csr_matrix

from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

In [2]:
# Function to process and reorganize dataset files.
    # INPUT: 
    #   filename: The root directory path of the original dataset
    #   datasetname: The target directory path where processed files will be stored
    # OUTPUT:
    #   Organizes and stores the processed files in the target directory, removing the first 4 lines from each file.
def file_iniate(filename, datasetname):
	dataset = filename
	for group in tqdm(os.listdir(dataset)):
		filepath = os.path.join(dataset, group)
		files = os.listdir(filepath)
		for file in files:
			with open(os.path.join(filepath, file), 'rb',) as f:
				lines=f.readlines()[4:]
	
			folder = os.path.join(datasetname, group)
			if not os.path.exists(folder):
				os.makedirs(folder)

			with open(os.path.join(folder, file), 'wb') as f:
				f.writelines(lines)
			
# Stratified Kfold Implement: Ensure that the proportion of categories in each fold is equal to 
#                             the proportion of categories in the original data source set
    # INPUT: 
    #   X: a list of data
    #   y: a list of labels
    #   k: kfold
    #   random_seed: fix randomness
    # OUTPUT: 
    #   yield a iterable generator, containing train indices and valid/test indices
def stratified_kfold(X, y, k=5, random_seed=None):
	if random_seed is not None:
		np.random.seed(random_seed)
	
	_, class_indices = np.unique(y, return_inverse=True)
	class_counts = defaultdict(list)
	
	for idx, class_idx in enumerate(class_indices):
		class_counts[class_idx].append(idx)

	folds = [[] for _ in range(k)]
	
	for class_idx, indices in class_counts.items():
		np.random.shuffle(indices)
		
		for i, idx in enumerate(indices):
			folds[i % k].append(idx)

	for i in range(k):
		test_indices = folds[i]
		train_indices = [idx for j in range(k) if j != i for idx in folds[j]]
		yield train_indices, test_indices
		
# Function to process a file and extract a list of words after cleaning.
def file2wordlist(filename):
	with open(filename, 'rb') as f:
		lines = f.readlines()
		
	string_lines = [line.decode('utf-8', errors='ignore').strip() for line in lines]
	cleaned_lines = [re.sub(r'[^a-zA-Z\s]', ' ', line.replace("'s", "")) for line in string_lines]
	result_string = ' '.join(cleaned_lines)
	words = re.findall(r'\b[a-zA-Z]+\b', result_string)

	words = list(map(lambda x: x.lower(), words))
	return words

# Function to count the frequency of words in a list of files.
def wordcount(files):
	words = []
	for f in tqdm(files):
		words.append(file2wordlist(f))
	return Counter([item for sublist in words for item in sublist]), words

In [3]:
# Call file_iniate function to process data from '20_newsgroups' to 'dataset', deleting first 4 lines and cleaning
files, dataset = '20_newsgroups', 'dataset'
file_iniate(files, dataset)

100%|██████████| 20/20 [00:05<00:00,  3.77it/s]


In [4]:
# Load data (file directory) and labels
data = []
labels = []
for i, group in enumerate(os.listdir(dataset)):
    filepath = os.path.join(dataset, group)
    files = os.listdir(filepath)
    data.extend([os.path.join(filepath, f) for f in files])
    labels.extend([i for _ in range(len(files))])

In [ ]:
# Function to compute the Term Frequency (TF) of terms in a document.
def compute_tf(document_tokens):
    term_count = Counter(document_tokens)
    total_terms = len(document_tokens)
    return {term: count / total_terms for term, count in term_count.items()}

# Function to compute the Inverse Document Frequency (IDF) of terms in a corpus.
def compute_idf(corpus_tokens):
    num_documents = len(corpus_tokens)
    idf = {}
    for document_tokens in corpus_tokens:
        for term in set(document_tokens):
            idf[term] = idf.get(term, 0) + 1
    return {term: np.log((num_documents + 1) / (1 + doc_count)) + 1 for term, doc_count in idf.items()}

# Function to compute the Term Frequency-Inverse Document Frequency (TF-IDF) for each document in the corpus.
def compute_tfidf(corpus_tokens):
    idf = compute_idf(corpus_tokens)
    tfidf_vectors = []
    for document_tokens in corpus_tokens:
        tf = compute_tf(document_tokens)
        tfidf_vectors.append({term: tf[term] * idf[term] for term in tf})
    return tfidf_vectors, idf

# Function to convert a list of TF-IDF vectors into a sparse matrix representation.
# This function uses the Compressed Sparse Row format to store the document-term matrix efficiently.
def vectorize(tfidf_vectors, idf):
    vocab = list(idf.keys())
    term_index = {term: idx for idx, term in enumerate(vocab)}

    num_docs = len(tfidf_vectors)
    num_terms = len(vocab)
    rows = []
    cols = []
    values = []

    for doc_idx, tfidf in enumerate(tfidf_vectors):
        for term, value in tfidf.items():
            if term in term_index:
                rows.append(doc_idx)
                cols.append(term_index[term])
                values.append(value)
    
    vectors = csr_matrix((values, (rows, cols)), shape=(num_docs, num_terms))
    return vectors

# Function to convert a list of documents into their vector representations. 
# idf is for vocabulary, necessary for Test set.
def doc2vec(documents, idf=None):
    if not idf:
        tfidf_vectors, idf = compute_tfidf(documents)
    else:
        tfidf_vectors, _ = compute_tfidf(documents)
    X = vectorize(tfidf_vectors, idf)
    return X, idf

In [6]:
acc = []
i = 1
for train_idx, test_idx in stratified_kfold(data, labels, k=5, random_seed=42):
    print(f'----------------------{i}th fold----------------------')
    i += 1

    train_files = [data[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]

    # train set
    wc, words = wordcount(train_files)
    stop = set(i for i, _ in wc.most_common(300))
    documents = [[i for i in sub if i not in stop] for sub in words]
    X, idf = doc2vec(documents)

    model = LinearSVC()
    model.fit(X, train_labels)

    # test set
    test_files = [data[i] for i in test_idx]
    test_labels = [labels[i] for i in test_idx]
    _, words = wordcount(test_files)
    documents = [[i for i in sub if i not in stop] for sub in words]
    X_test, _ = doc2vec(documents, idf)

    y_test = model.predict(X_test)
    print(classification_report(y_test, test_labels))
    acc.append(accuracy_score(y_test, test_labels))
    

--------------1th fold--------------


100%|██████████| 4000/4000 [00:13<00:00, 300.65it/s]


              precision    recall  f1-score   support

           0       0.78      0.77      0.78       202
           1       0.83      0.86      0.85       195
           2       0.86      0.79      0.83       218
           3       0.77      0.85      0.81       182
           4       0.88      0.91      0.90       193
           5       0.88      0.92      0.90       191
           6       0.88      0.89      0.89       197
           7       0.94      0.88      0.91       214
           8       0.97      0.97      0.97       200
           9       0.96      0.96      0.96       199
          10       0.97      0.97      0.97       202
          11       0.97      0.97      0.97       202
          12       0.85      0.88      0.87       194
          13       0.97      0.96      0.96       203
          14       0.96      0.96      0.96       201
          15       0.99      0.98      0.99       204
          16       0.93      0.87      0.90       212
          17       0.95    

100%|██████████| 4000/4000 [00:00<00:00, 5528.77it/s]


              precision    recall  f1-score   support

           0       0.76      0.81      0.78       186
           1       0.89      0.86      0.87       208
           2       0.80      0.88      0.84       182
           3       0.83      0.82      0.83       204
           4       0.91      0.88      0.90       206
           5       0.92      0.90      0.91       203
           6       0.81      0.83      0.82       197
           7       0.91      0.92      0.92       197
           8       0.96      0.93      0.94       207
           9       0.95      0.98      0.97       194
          10       0.98      0.96      0.97       204
          11       0.98      0.98      0.98       202
          12       0.84      0.88      0.86       192
          13       0.97      0.97      0.97       202
          14       0.95      0.95      0.95       200
          15       1.00      0.97      0.99       206
          16       0.92      0.86      0.89       215
          17       0.96    

...

              precision    recall  f1-score   support

           0       0.73      0.76      0.75       194
           1       0.84      0.84      0.84       199
           2       0.84      0.85      0.85       199
           3       0.80      0.79      0.80       202
           4       0.86      0.90      0.88       193
           5       0.88      0.91      0.89       194
           6       0.84      0.89      0.86       189
           7       0.92      0.91      0.91       201
           8       0.97      0.97      0.97       199
           9       0.94      0.95      0.95       198
          10       0.98      0.97      0.98       203
          11       0.97      0.95      0.96       205
          12       0.87      0.84      0.85       208
          13       0.89      0.94      0.91       188
          14       0.97      0.92      0.94       211
          15       0.99      0.99      0.99       199
          16       0.87      0.86      0.86       203
          17       0.96    

100%|██████████| 3999/3999 [00:00<00:00, 5663.72it/s]


              precision    recall  f1-score   support

           0       0.75      0.75      0.75       201
           1       0.83      0.85      0.84       195
           2       0.77      0.82      0.80       187
           3       0.82      0.78      0.80       211
           4       0.89      0.92      0.90       194
           5       0.90      0.89      0.89       202
           6       0.83      0.84      0.83       198
           7       0.91      0.96      0.94       189
           8       0.97      0.96      0.96       203
           9       0.95      0.95      0.95       200
          10       0.97      0.98      0.98       198
          11       0.97      0.97      0.97       201
          12       0.93      0.89      0.91       209
          13       0.95      0.94      0.95       202
          14       0.95      0.95      0.95       200
          15       1.00      0.97      0.99       205
          16       0.89      0.89      0.89       200
          17       0.94    

100%|██████████| 3999/3999 [00:00<00:00, 4371.52it/s]


              precision    recall  f1-score   support

           0       0.84      0.76      0.80       221
           1       0.86      0.83      0.85       207
           2       0.90      0.87      0.88       205
           3       0.74      0.85      0.79       175
           4       0.91      0.91      0.91       198
           5       0.90      0.91      0.91       197
           6       0.88      0.86      0.87       205
           7       0.93      0.95      0.94       194
           8       0.97      0.97      0.97       201
           9       0.95      0.95      0.95       202
          10       0.97      0.96      0.97       203
          11       0.96      0.95      0.96       203
          12       0.90      0.87      0.88       206
          13       0.96      0.94      0.95       205
          14       0.96      0.96      0.96       202
          15       1.00      1.00      1.00       199
          16       0.91      0.88      0.89       206
          17       0.95    

In [7]:
print(f'Average Accuracy is {sum(acc)/len(acc)}')

Average Accuracy is 0.8828320830207552
